<p>
    <img src="https://s3.amazonaws.com/iotanalytics-templates/Logo.png" style="float:left;">
    <h1 style="color:#1A5276;padding-left:115px;padding-bottom:0px;font-size:28px;">Serverless Industry 4.0 & AI: Drive Business Insights from Machine Data</h1>
</p>
<p style="color:#1A5276;padding-left:117px;position:absolute;margin-top:-10px;font-style:italic;font-size:18px;padding-bottom:20px">
By Markus Bestehorn</p>
<p style="clear:all"><br/></p>
<p>This notebook is used for the re:invent 2018 workshop IOT402. This Notebook will use sample data to create a ML model that can be deployed on AWS Greengrass with AWS Greengrass ML Inference.</p>

<p>We'll make use of `gluon.nn.Sequential` to build a neural network that classifies data into two classes: Abnormal drilling processes and normal drilling processes.</p>


# Imports & Setup
First we'll import the necessary bits.

In [86]:
from __future__ import print_function
import numpy as np
import mxnet as mx
from mxnet import nd, autograd, gluon
import logging
import sys, os
import pandas as pd
import copy
import json
import time
import boto3
import tarfile
logger = logging.getLogger(__name__)
logging.basicConfig(stream=sys.stdout, level=logging.INFO)



We'll also want to set the contexts for our data and our models. MXNET can use GPUs for preprocessing data as well as training and inference. In general, GPUs will perform better for these tasks as the underlying  mathematical operations for these steps are faster on a GPU. Since this example is fairly small and does not require huge amounts of data, you can still use a CPU. The following code will determine if MXNET can you a GPU in the following.

In [87]:
from subprocess import CalledProcessError
ctx = None
try:
    if (mx.test_utils.list_gpus()):
        logger.info("Using GPU")
        ctx = mx.gpu() 
    else:
        ctx = mx.cpu()
        logger.info("Using CPU")
except CalledProcessError:
    ctx = mx.cpu()
    logger.info("Using CPU")
finally:
    data_ctx = ctx
    model_ctx = ctx
    logger.info(model_ctx)


INFO:__main__:Using CPU
INFO:__main__:cpu(0)


The following step is ***optional***: Some of the steps in the following have a random component. For instance, the initialization of the neural network will be done by assigning random values to the parameters. The following step will set the random value generator seeds to constant values. This means the random generators will always generate the same sequence of numbers and therefore the execution of the notebook will always result in exactly the same result, which may be desirable from a reproduction point of view. If you think that you can cope with some of the values in the following having slightly different values, feel free to skip this step.

In [88]:
mx.random.seed(42)


# Loading the dataset
This section explores the data set and gives some insights into their structure. First, we will load a set of header names that the CSV files use. These make it easier to access the data by (column) name. The last part of the following code defines three sets of attributes:
* The observation features contains only a single attribute which is the `HasAnomaly` attribute of the input data. This value indicates whether the corresponding drilling process was "normal" (0) or "abnomal" (1). This is the value we want to predict.
* The train attributes are all the attributes that contain sensor values or any other values that are measures. We will use these features to train a model. Note that these features DO NOT contain the observation.
* The viewing attributes are the union of the two aforementioned attibute sets and only used for viewing purposes, so we can see data more easily in a single view. This set has no functional value.

In [89]:
ANOMALY_FLAG_KEY = "HasAnomaly"
AVG_PRESSURE_KEY = "AvgPressure"
AVG_MOTOR_KEY = "AvgMotor"
AVG_SPINDLE_KEY = "AvgSpindle"
DURATION_KEY = "duration"
WORK_START_TIMESTAMP_KEY = "WorkStartTimestamp"
WORK_END_TIMESTAMP_KEY = "WorkEndTimestamp"
WORK_START_TIME_KEY = "WorkStart"
WORK_END_TIME_KEY = "WorkEnd"
ANOMALY_DURATION_LONG_KEY = "DurationTooLong"
ANOMALY_DURATION_SHORT_KEY = "DurationTooShort"
ANOMALY_PRESSURE_HIGH_KEY = "PressureHigh"
ANOMALY_PRESSURE_LOW_KEY = "PressureLow"
ANOMALY_MOTOR_HIGH_KEY = "MotorHigh"
ANOMALY_MOTOR_LOW_KEY = "MotorLow"
ANOMALY_SPINDLE_HIGH_KEY = "SpindleHigh"
ANOMALY_SPINDLE_LOW_KEY ="SpindleLow"


CSV_HEADERS = []
CSV_HEADERS.append(ANOMALY_FLAG_KEY)
CSV_HEADERS.append(AVG_PRESSURE_KEY)
CSV_HEADERS.append(AVG_MOTOR_KEY)
CSV_HEADERS.append(AVG_SPINDLE_KEY)
CSV_HEADERS.append(DURATION_KEY)
CSV_HEADERS.append(WORK_START_TIMESTAMP_KEY)
CSV_HEADERS.append(WORK_END_TIMESTAMP_KEY)
CSV_HEADERS.append(WORK_START_TIME_KEY)
CSV_HEADERS.append(WORK_END_TIME_KEY)
CSV_HEADERS.append(ANOMALY_DURATION_LONG_KEY)
CSV_HEADERS.append(ANOMALY_DURATION_SHORT_KEY)
CSV_HEADERS.append(ANOMALY_PRESSURE_HIGH_KEY)
CSV_HEADERS.append(ANOMALY_PRESSURE_LOW_KEY)
CSV_HEADERS.append(ANOMALY_MOTOR_HIGH_KEY)
CSV_HEADERS.append(ANOMALY_MOTOR_LOW_KEY)
CSV_HEADERS.append(ANOMALY_SPINDLE_HIGH_KEY)
CSV_HEADERS.append(ANOMALY_SPINDLE_LOW_KEY)

OBSERVATION_FEATURES = [ANOMALY_FLAG_KEY]
TRAIN_ATTRIBUTES = [AVG_PRESSURE_KEY, AVG_MOTOR_KEY, AVG_SPINDLE_KEY, DURATION_KEY]
VIEWING_ATTRIBUTES = OBSERVATION_FEATURES + TRAIN_ATTRIBUTES

TRAIN_FEATURE_COUNT = len(TRAIN_ATTRIBUTES)

Load the data from the CSV into and print some details about the data.

In [90]:
# specify dataset
dataset = "classified_data"
# create IoT Analytics client
iota_client = boto3.client('iotanalytics')

# import target Data Set from AWS IoT Analytics service
try:
    dataset_url = iota_client.get_dataset_content(datasetName = dataset)['entries'][0]['dataURI']
    data_all = pd.read_csv(dataset_url)
    if data_all.empty:
        raise Exception('No data found in AWS IoT Analytics data set - can you please ensure you have data and clicked Actions->Run Now in the data set? ')
    # iot analytics stores all column names as lowercase
    VIEWING_ATTRIBUTES = [x.lower() for x in VIEWING_ATTRIBUTES]
    TRAIN_ATTRIBUTES = [x.lower() for x in TRAIN_ATTRIBUTES]
    OBSERVATION_FEATURES = [x.lower() for x in OBSERVATION_FEATURES]
    CSV_HEADERS = [x.lower() for x in CSV_HEADERS]
    dd_all = data_all[[x.lower() for x in VIEWING_ATTRIBUTES]]
    dd_features = data_all[[x.lower() for x in TRAIN_ATTRIBUTES]]
    dd_observation = data_all[[x.lower() for x in OBSERVATION_FEATURES]]
    #display(data_all)
    logger.info("loaded data from AWS IoT Analytics dataset '%s'", dataset)
    if data_all.empty:
        raise Exception('No data found')
# use backup dataset if dataset not found
except Exception as e:
    logger.error("Cannot load dataset '%s' from AWS IoT Analytics - use generic csv file ", dataset)
    logger.error(e)
    data_all = pd.read_csv('https://s3-eu-west-1.amazonaws.com/industrial-architecture-workshop-eu-west-1/2020-06-03/sagemaker/Drill-Data-All.csv')
    dd_all = data_all[VIEWING_ATTRIBUTES]
    dd_features = data_all[TRAIN_ATTRIBUTES]
    dd_observation = data_all[OBSERVATION_FEATURES]
finally:
    logger.info("Shape of complete matrix: " + str(np.shape(dd_all)))
    logger.info("Shape of feature matrix: " + str(np.shape(dd_features)))
    logger.info("Shape of observation: " + str(np.shape(dd_observation)))
    #display(dd_all) # uncomment this line to see the data in detail (optional)
    dd_all_description = dd_all.describe() # store the data description
    dd_all.describe() # print the description of the data for viewing

INFO:__main__:loaded data from AWS IoT Analytics dataset 'classified_data'
INFO:__main__:Shape of complete matrix: (7364, 5)
INFO:__main__:Shape of feature matrix: (7364, 4)
INFO:__main__:Shape of observation: (7364, 1)


# Preprocessing the data set
We have to normalize the attributes to [0,1] range to avoid numerical problems during the learning process. Note that the normalization is only done to the features, but not to the observations. The observations are already 0,1-values.

Note that this step can take a while depending on the performance of the underlying hardware! The code has been written with readibility in mind and therefore does not use a vectorized implementation, but instead a more easily understandable "looped" implementation. For larger data sets, it is highly recommended to use a vectorized implementation.

In [91]:
# suppress a confusing warning about writing to a data frame.
pd.options.mode.chained_assignment = None  # default='warn'

def normalizeFeatureValue(value, min_val, min_max_diff):
    return (value - min_val) / min_max_diff

def normalizeAttribute(inputData, description, min_val=None, max_val=None):
    if (min_val is None):
        min_val = float(description["min"])
    if (max_val is None):
        max_val = float(description["max"])
    diff = max_val - min_val
    for index in range(0,len(inputData)):
        inputData[index] = normalizeFeatureValue(value=inputData[index], min_val=min_val, min_max_diff=diff)
    return inputData

def normalize(data, attributes=TRAIN_ATTRIBUTES, description=None):
    if (description is None):
        description = data.describe()
    # make a deep copy to avoid reference bugs
    resultData = copy.deepcopy(data)
    # normalize by attribute
    for attr in attributes:
        #logger.info("\n**********\nNormalizing input data of attribute " + str(attr) + "\n**********")
        normalizeAttribute(resultData[attr], description[attr])
    
    return resultData

Finally, we can run these normalization functions as follows:

In [92]:
startTime = time.time()
dd_all_norm = normalize(dd_all)
endTime = time.time()
dd_all_norm.describe()

loopedTime = endTime - startTime
logger.info("Normalization took " + str(loopedTime) + " seconds.")

INFO:__main__:Normalization took 3.13994598389 seconds.


### OPTIONAL: Vectorization
The following step is ***optional*** and uses a vectorized implementation of the normalization step. There is only a single loop over the different attributes. Contrary to the looped version of the normalization, this has only a single line. Looking carefully at the data types, the line has a mixed set of operands: While `resultData[attr]` is a vector containing all the values of the current attribute `attr`, the other operands are scalars (floats). The vectorized and highly optimized operators that allow the subtraction/division of a vector by a scalar are the foundation of the speed improvement that we will be able to see when executing this step.

In [93]:
def normalizeVectorized(data, attributes=TRAIN_ATTRIBUTES, description=None):
    if (description is None):
        description = data.describe()
    # make a deep copy to avoid reference bugs
    resultData = copy.deepcopy(data)
    # normalize by attribute
    for attr in attributes:
        min_val = float(description[attr]["min"])
        max_val = float(description[attr]["max"])
        
        # vectorized normalization implementation
        resultData[attr] = (resultData[attr] - min_val) / (max_val - min_val)
    return resultData

startTime = time.time()
vectorizedResult = normalizeVectorized(dd_all)
endTime = time.time()
vectorizedTime = endTime - startTime

logger.info("Vectorized version took " + str(vectorizedTime) + " sec. compared to " + str(loopedTime) + " sec. with loops.")
logger.info("Vectorization has an improvement of " + str(loopedTime / vectorizedTime * 100) + "% over the loop implementation")

vectorizedResult.describe()

INFO:__main__:Vectorized version took 0.0219140052795 sec. compared to 3.13994598389 sec. with loops.
INFO:__main__:Vectorization has an improvement of 14328.4896751% over the loop implementation


,hasanomaly,avgpressure,avgmotor,avgspindle,duration
count,7364.000000,7364.000000,7364.000000,7364.000000,7364.000000
mean,0.317219,0.213074,0.439226,0.538595,0.292958
std,0.465425,0.223895,0.184287,0.176701,0.225032
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.159601,0.419988,0.531790,0.121045
50%,0.000000,0.161062,0.422786,0.534343,0.239796
75%,1.000000,0.162497,0.425598,0.536965,0.357423
max,1.000000,1.000000,1.000000,1.000000,1.000000


The result of the computation is exactly the same as in the looped implementation. A typical result of the execution is as follows:
>`INFO:__main__:Vectorized version took 0.0220000743866 sec. compared to 0.638000011444 sec. with loops.
INFO:__main__:Vectorization has an improvement of 2899.99024655% over the loop implementation`

This means even for this small data set, the vectorized implementation was about 28-times faster than the looped implementation. This gap widens as soon as datasets grow bigger and fail to fit into main memory.

***Learning***: If preprocessing is slow, check if everything has been vectorized.

## Training & Test Data Sets
As shown above, the data has now been normalized to values of the range [0,1]. Next, we split the data in the features X and the labels y and turn them into nd.array instances.

In [94]:
X = nd.array(dd_all_norm[TRAIN_ATTRIBUTES])
y = nd.array(dd_all_norm[OBSERVATION_FEATURES])
display(X)
display(y)


[[0.15930896 0.42935792 0.5381709  0.19737233]
 [0.15955013 0.4175472  0.53268063 0.07995133]
 [0.1564667  0.4206642  0.53089744 0.27804887]
 ...
 [0.16155158 0.42147252 0.5355038  0.12083457]
 [0.9866026  0.00840182 0.53304225 0.9121569 ]
 [0.16119407 0.00765268 0.5329813  0.27758214]]
<NDArray 7364x4 @cpu(0)>


[[0.]
 [0.]
 [0.]
 ...
 [0.]
 [1.]
 [1.]]
<NDArray 7364x1 @cpu(0)>

Let´s measure the shapes of the two matrices X and y and store some information about them so we can use it later on.

In [95]:
X_shape = X.shape
y_shape = y.shape
totalSampleCount = X_shape[0]
inputFeatureCount = X_shape[1]
logger.info("Each dataset in X has " + str(inputFeatureCount) + " features and there are " + str(totalSampleCount) + " samples.")

INFO:__main__:Each dataset in X has 4 features and there are 7364 samples.


Next we need to define the training and the test data set. The training set will be used to train the model. The test data set will be used to determine how well the model generalizes over previously unseen data.

We use 80% of the samples for training and 20% for the test set.

In [96]:
num_outputs = 2
train_test_ratio = 0.8


trainSampleCount = int(totalSampleCount*train_test_ratio)
testSampleCount = int(totalSampleCount - trainSampleCount)
logger.info("Expecting the training set to have " + str(trainSampleCount) + " samples.")
logger.info("Expecting the test set to have " + str(testSampleCount) + " samples.")
logger.info("Given that the total number of samples is " + str(totalSampleCount) + ", this is expected.")

train_dset = mx.gluon.data.dataset.ArrayDataset(X[:trainSampleCount, :], y[:trainSampleCount])
test_dset = mx.gluon.data.dataset.ArrayDataset(X[trainSampleCount:, :], y[trainSampleCount:])

# get a sample from the dataset
index = 5
sample_X = train_dset[index][0] # get item at index 5 and 0 => 0 because X is the first part of the data set
sample_y = train_dset[index][1] # get item at index 5 and 1 => 0 because y is the first part of the data set
logger.info("The X portion of the sample is:\n" + str(sample_X) + "\n\n")
logger.info("The y portion of the sample is:\n" + str(sample_y))

INFO:__main__:Expecting the training set to have 5891 samples.
INFO:__main__:Expecting the test set to have 1473 samples.
INFO:__main__:Given that the total number of samples is 7364, this is expected.
INFO:__main__:The X portion of the sample is:

[0.9790082  0.98531234 0.5332652  0.27802882]
<NDArray 4 @cpu(0)>


INFO:__main__:The y portion of the sample is:

[1.]
<NDArray 1 @cpu(0)>


## Data Loaders
In this section, we create two data loaders: One contains the training data and the other one contains the test data. The third one created here is just for viewing purposes to see what kind of data will be produced by the loader.
The loader will provide always two pieces - features X (X_batch) and a corresponding label (y_batch) -  and each batch contains 64 samples (as defined by batch_size). The shapes printed by the for loop at the end of this piece of code should reflect that.

In [97]:
batch_size = 64 # usually defined so that one batch fits into memory. Not really relevant for this example.

# create a data loader for the sets
train_loader = mx.gluon.data.DataLoader(train_dset, batch_size=batch_size)
test_loader = mx.gluon.data.DataLoader(test_dset, batch_size=batch_size)
show_loader = mx.gluon.data.DataLoader(train_dset, batch_size=batch_size)

# show how the loader works
count = 0
for X_batch, y_batch in show_loader:
    print("X_batch has shape {}, and y_batch has shape {}".format(X_batch.shape, y_batch.shape))
    count += 1
    if (count > 2):
        break

X_batch has shape (64, 4), and y_batch has shape (64, 1)
X_batch has shape (64, 4), and y_batch has shape (64, 1)
X_batch has shape (64, 4), and y_batch has shape (64, 1)


# Building the Neural Network

For this example, we will use a "very boring" neural network (NN) architecture. It will just chain 3 layers together and use a relu activation function. All nodes in each layer are connected to all nodes in the following layer (dense network).
Given the drill data characteristics, this network should be sufficient to make accurate predictions whether a given data set is an anomaly or not.

Note that adding more layers or using other activation functions is rather simple due to Gluon's `Sequential` class.

In [98]:
num_hidden = 4
# this function returns a neural network based on the corresponding gluon sequential class
# use gluon.nn.Sequential() to build a normal sequential model
# use gluon.nn.HybridSequential() to build a hybrid model. The hybrid model can be saved along with the parameters
def buildNeuralNetwork(net=gluon.nn.Sequential()):
    with net.name_scope():
        net.add(gluon.nn.Dense(num_hidden, activation="relu"))
        net.add(gluon.nn.Dense(num_hidden, activation="relu"))
        net.add(gluon.nn.Dense(num_outputs))
    return net

def buildHybridNeuralNetwork():
    net = buildNeuralNetwork(net=gluon.nn.HybridSequential())
    net.hybridize()
    return net

net = buildHybridNeuralNetwork()

## Parameter initialization
Based on this network definition, we have to provide initial values for the parameters of the NN. Gluon also has a nice set of functions and classes that takes care of that. Basically, this line assigns random values drawn from a normal distribution.


In [99]:
net.collect_params().initialize(mx.init.Normal(sigma=.1), ctx=model_ctx, force_reinit=True)

## Softmax cross-entropy loss
It is mandatory for the optimizer to have a loss function to determine how "far" away from the correct predictions it was. Again, Gluon has a set of these loss functions already implemented, so we can just use one. In this case, SoftmaxCrossEntropyLoss is the choice.

In [100]:
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()

## Optimizer
This line defines a trainer - basically Gluons terminology for the class that runs the training. The optimizer we use here is called "Adam", but we an also use Single Gradient Descent (abbreviated "sgd") as well.

In [101]:
learning_rate = 0.001
trainer = gluon.Trainer(net.collect_params(), 'adam', {'learning_rate': learning_rate})
#trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': learning_rate})


## Evaluation metric
Next, we define a functiont that will use the NN and some data loader to determine the accuracy. This will allow us to determine the accuracy on the train as well as the test set by simply passing this function a different loader. The metric to compute the accuracy is provided as a class by Gluon.

In [102]:
def evaluate_accuracy(data_iterator, net):
    acc = mx.metric.Accuracy() # get the Gluon metric to compute accuracy for classification
    for data, label in data_iterator: # use the data in the data loader to run them through the evaluation
        data = data.as_in_context(model_ctx).reshape((-1, inputFeatureCount)) # here we load the features
        label = label.as_in_context(model_ctx) # here we get the corresponding labels
        output = net(data) # this is where we run the features through the network to make predictions
        '''
        The next line is a bit more complicated so we explain it in detail here:
        The output of the NN are two values for each input dataset - the first is the probability that the dataset belongs to
        the first class and the second one is the probability that it belongs to the second class. 
        '''
        predictions = nd.argmax(output, axis=1).reshape((-1, 1))
        acc.update(preds=predictions, labels=label)
    return acc.get()[1]

## Training loop
This is the main loop that runs through the training data to train the model. At the end of each epoch (once it has run through the training data once), it will evaluate the model by measuring the accuracy on the test and the train set.
The accuracy of the model on the test set is what we are looking for, because the higher this accuracy is, the better is the model. This is because the test data contains previously unseen data, i.e., the prediction is made on data that the model has not seen during training.

In [103]:
epochs = 120

for e in range(epochs):
    cumulative_loss = 0
    for (data, label) in train_loader:
        data = data.as_in_context(model_ctx).reshape((-1, inputFeatureCount))
        label = label.as_in_context(model_ctx)
        with autograd.record():
            output = net(data)
            loss = softmax_cross_entropy(output, label)
        loss.backward()
        trainer.step(data.shape[0])
        cumulative_loss += nd.sum(loss).asscalar()


    test_accuracy = evaluate_accuracy(test_loader, net)
    train_accuracy = evaluate_accuracy(train_loader, net)
    if (e % 10 == 0):
        print("Epoch %s. Loss: %s, Train_acc %s, Test_acc %s" %
              (e, cumulative_loss, train_accuracy, test_accuracy))

Epoch 0. Loss: 3980.340631008148, Train_acc 0.683754880326, Test_acc 0.678886625933
Epoch 10. Loss: 1487.996562719345, Train_acc 0.934306569343, Test_acc 0.928716904277
Epoch 20. Loss: 622.5797629356384, Train_acc 0.97657443558, Test_acc 0.974202308215
Epoch 30. Loss: 425.7857804298401, Train_acc 0.98081819725, Test_acc 0.978954514596
Epoch 40. Loss: 242.20052921772003, Train_acc 0.984382957053, Test_acc 0.983706720978
Epoch 50. Loss: 87.65450006723404, Train_acc 0.999320998133, Test_acc 1.0
Epoch 60. Loss: 28.776299942284822, Train_acc 1.0, Test_acc 1.0
Epoch 70. Loss: 12.073033042252064, Train_acc 1.0, Test_acc 1.0
Epoch 80. Loss: 5.882756280712783, Train_acc 1.0, Test_acc 1.0
Epoch 90. Loss: 3.103112092707306, Train_acc 1.0, Test_acc 1.0
Epoch 100. Loss: 1.7146259879227728, Train_acc 1.0, Test_acc 1.0
Epoch 110. Loss: 0.9753940722439438, Train_acc 1.0, Test_acc 1.0


Once this code has finished executing, the training of the NN has finished. The two last lines of output should look similar to the following:
>`
Epoch 100. Loss: 235.31822562217712, Train_acc 0.958559782609, Test_acc 0.953929539295
Epoch 110. Loss: 190.30429339408875, Train_acc 0.968070652174, Test_acc 0.956639566396
`

This means the accuracy over the training set, i.e., data that has been used for training, is almost 97%. Given the simplicity of the NN and the small size of the data set, this is already a good result. To check how the model performs with previously unseen data, i.e., data that has not been used for training, the evaluation also uses the training set. The output above means that our model achieves about 96% accuracy on this unknown data set.

To avoid lengthy training & wait times, the training has been limited to 120 epochs, i.e., the training loop is executed 120 times. Looking at the loss shown in the output, we can see that it was still declining after the 120th epoch. This can indicate that continuing the training may yield better results. For the purposes of this notebook, results similar to the ones above are sufficient. Hence, it is ***optional*** to re-execute the training loop again and see if the accuracy improves.

# Making predictions
This section shows how to use the model to make predictions using some artifical examples. This allows us to check whether the model will find anomalies that we expect to be detected.

## An "Average" Drilling procedure
First, let´s define an artificial piece of data using the mean values of the input data that we had. The following code creates this artificial data set and also collects the min/max values of the corresponding attributes. These values are needed to normalize the input later on.

In [104]:
features = []
min_values = []
max_values = []
for attr in TRAIN_ATTRIBUTES:
    features.append(dd_all_description[attr]["mean"])
    min_values.append(dd_all_description[attr]["min"])
    max_values.append(dd_all_description[attr]["max"])

print("The min-values are: " + str(min_values))    
print("The max-values are: " + str(max_values))

display(features)

The min-values are: [7.815327713140493, 849.0821682136137, 551.0287156231697, 10.01059556007385]
The max-values are: [24.4976786018849, 2223.4170737832915, 2158.418250804427, 22.64933466911316]


[11.369910665017434,
 1452.7263457731235,
 1416.7602756228212,
 13.713214136089467]

As we can see above, the loop has collected the attribute values as a list and we can use this list of artificial attribute values as an input for our prediction. Note that these values are not normalized yet!

In [105]:
avgData = nd.array(features)
display(avgData)
output = net(avgData.reshape((-1, inputFeatureCount)))
display(output)
predictions = nd.argmax(output, axis=1)
display(predictions)


[  11.36991  1452.7263   1416.7603     13.713214]
<NDArray 4 @cpu(0)>


[[-20035.268  19389.29 ]]
<NDArray 1x2 @cpu(0)>


[1.]
<NDArray 1 @cpu(0)>

The output above has three parts: The first part is the result of the transformation of the list into an nd-array. The values have remained the same. The second part is the output of the NN which has been generated when the artificial data has been passed into the NN. In one of our executions, this yielded the following results:
>`
[[-12494.348  12190.844]]
<NDArray 1x2 @cpu(0)>
`

The actual values may look different. Normally, the values are probabilities returned by the NN which indicate whether the input data corresponds to a normal drilling process or a non-normal one. The reason why these two values are not probabilities and hence do not yield a proper result is the missing normalization of the input.

The code below changes this: The input generated above is normalized using the same function we have applied to the input data. Once this is done, we pass the normalized data into the NN again and the result is as expected: A set of average sensor values is labelled as a "normal" drilling process.

In [106]:
for index in range(0, inputFeatureCount):
    diff = max_values[index] - min_values[index]
    avgData[index] = normalizeFeatureValue(avgData[index], min_val=min_values[index], min_max_diff=diff)
display(avgData)

output = net(avgData.reshape((-1, inputFeatureCount)))
display(output)
predictions = nd.argmax(output, axis=1)
display(predictions)


[0.21307445 0.4392264  0.5385948  0.29295793]
<NDArray 4 @cpu(0)>


[[ 5.7097406 -5.175905 ]]
<NDArray 1x2 @cpu(0)>


[0.]
<NDArray 1 @cpu(0)>

Let´s have a closer look at the result and interpret it a bit. Our execution yielded a result as follows:
>`
[[ 1.7810409 -1.7810413]]
<NDArray 1x2 @cpu(0)>
[0.]
<NDArray 1 @cpu(0)>
`

This means the NN predicted that the input corresponds to a normal drilling process. The next piece of code wraps the whole procedure of normalizing the input and making a prediction on it into a set of functions.

In [107]:
# Note: the following two functions can also be used to make predictions on multiple input data set
# Therefore, both return arrays, more specifically, their return values are instances of nd.array
def __makePrediction(normalizedInputData, net=net):
    output = net(normalizedInputData.reshape((-1, inputFeatureCount)))
    return nd.argmax(output, axis=1)

def makePrediction(rawInputData, minValues=min_values, maxValues=max_values, net=net):
    if (len(rawInputData) != inputFeatureCount):
        raise IllegalArgumentException("Out model only has " + str(inputFeatureCount) + " inputs. You cannot use more.")
    data = nd.array(rawInputData) # make sure the input is nd.array
    # normalize
    for index in range(0, len(rawInputData)):
        diff = max_values[index] - min_values[index]
        data[index] = normalizeFeatureValue(data[index], min_val=min_values[index], min_max_diff=diff)
    # make a prediction
    return __makePrediction(normalizedInputData=data, net=net)

# This method makes a prediction on a single set of sensor values and returns a single int value
def predict(pressure, motor, spindle, duration, minValues=min_values, maxValues=max_values, net=net):
    # generate a list of feature values
    features = [pressure, motor, spindle, duration]
    # make a prediction
    r = makePrediction(rawInputData=features, minValues=minValues, maxValues=maxValues, net=net)
    return int(r.asscalar())

# This method makes a prediction on a single set of sensor values and returns TRUE if it is a normal dataset 
# and otherwise False.
def isNormal(pressure, motor, spindle, duration, minValues=min_values, maxValues=max_values, net=net):
    if (predict(pressure=pressure, motor=motor, spindle=spindle, duration=duration, minValues=minValues, maxValues=maxValues, net=net) == 0):
        return True
    else:
        return False
    
# make some example predictions
logger.info("Numerical prediction result: " + str(makePrediction(features)) + "\n") # should return 0 as above
logger.info("Boolean prediction result: " + str(isNormal(1,2,3,4)))

INFO:__main__:Numerical prediction result: 
[0.]
<NDArray 1 @cpu(0)>

INFO:__main__:Boolean prediction result: False


## An Exceptional Drilling Procedure
Next, we create a sample that must be detected as an anomaly using the max values for each attribute.

In [108]:
maxFeatures = []
for attr in TRAIN_ATTRIBUTES:
    maxFeatures.append(dd_all_description[attr]["max"])
    
display(maxFeatures)

makePrediction(rawInputData=maxFeatures)

[24.4976786018849, 2223.4170737832915, 2158.418250804427, 22.64933466911316]


[1.]
<NDArray 1 @cpu(0)>

# Storing & Exporting the model
To make use of the model in an IoT context, we have to export it and deploy it on edge devices. This requires the following steps:
* Exporting the model and storing it to a file along with the min/max values required for normalization of the data
* Uploading the files to S3
* Deploy the model on edge devices
This notebook only addresses the first two items of this list. The actual deployment on edge devices is outside of the scope of this notebook.

## Export
The export of the NN and all relevant parts for predictions requires 3 files: One file contains the structure of the network, i.e., a description of the layers and their structure. The second file contains the set of parameters that are used for the prediction and have been learned during the training phase. Finally, the third file has the variables needed to normalize input with the corresponding min-max-values that have been used for the normalization of the training data.

First, we define a couple of names that will allow us easier access to the files later on.

In [109]:
filePrefix="Drill-AnomalyClassifier"
epoch = 0
paramFile = filePrefix + "-0000" + ".params"
structureFile = filePrefix + "-symbol.json"
normalizationFile = filePrefix + "-normalization.json"

The next line uses Gluon functionality to write 1) the network structure and 2) the NN parameters derived during training. The network structure is stored as JSON in a file whose name is generated from the given prefix and "-symbol.json". The parameters are stored in the ".params" file using the prefix as well.

In [110]:
# this line does the export of the parameters and the NN architecture/structure
net.export(filePrefix, epoch=epoch)

Optionally, we can load the file and have a look at the content.

In [111]:
logger.info("Loading structure file with name " + str(structureFile))
with open(structureFile) as f:
    nn_json = json.load(f)
display(nn_json)

INFO:__main__:Loading structure file with name Drill-AnomalyClassifier-symbol.json


{u'arg_nodes': [0, 1, 2, 5, 6, 9, 10],
 u'attrs': {u'mxnet_version': [u'int', 10600]},
 u'heads': [[11, 0, 0]],
 u'node_row_ptr': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
 u'nodes': [{u'inputs': [], u'name': u'data', u'op': u'null'},
  {u'attrs': {u'__dtype__': u'0',
    u'__lr_mult__': u'1.0',
    u'__shape__': u'(4, 0)',
    u'__storage_type__': u'0',
    u'__wd_mult__': u'1.0'},
   u'inputs': [],
   u'name': u'hybridsequential3_dense0_weight',
   u'op': u'null'},
  {u'attrs': {u'__dtype__': u'0',
    u'__init__': u'zeros',
    u'__lr_mult__': u'1.0',
    u'__shape__': u'(4,)',
    u'__storage_type__': u'0',
    u'__wd_mult__': u'1.0'},
   u'inputs': [],
   u'name': u'hybridsequential3_dense0_bias',
   u'op': u'null'},
  {u'attrs': {u'flatten': u'True', u'no_bias': u'False', u'num_hidden': u'4'},
   u'inputs': [[0, 0, 0], [1, 0, 0], [2, 0, 0]],
   u'name': u'hybridsequential3_dense0_fwd',
   u'op': u'FullyConnected'},
  {u'attrs': {u'act_type': u'relu'},
   u'inputs': [[3, 0, 0]],

As a final step, we create a dictionary that contains the variables needed to do normalization of input data. Using the standard methods for writing a JSON file, can store them on disk.

In [112]:
trainAttributeKey = "train-attributes"
observationKey = "observation-attribute"
minValuesKey = "min-values"
maxValuesKey = "max-values"

normalizationDict = {}
normalizationDict[trainAttributeKey] = TRAIN_ATTRIBUTES
normalizationDict[observationKey] = OBSERVATION_FEATURES
normalizationDict[minValuesKey] = min_values
normalizationDict[maxValuesKey] = max_values

with open(normalizationFile, "w") as f:
    json.dump(normalizationDict, f)

## Loading the model from file
This section demonstrates what needs to happen on the device to reload the model and get a neural network back.

In [113]:
with open(normalizationFile) as f:
    newNormalizationDict = json.load(f)
newTrainAttributes = newNormalizationDict[trainAttributeKey]
newObservationAttribute = newNormalizationDict[observationKey]
newMinValues = newNormalizationDict[minValuesKey]
newMaxValues = newNormalizationDict[maxValuesKey]

logger.info("Loaded Training attrs:\t" + str(newTrainAttributes))
logger.info("Loaded Observation attr:\t" + str(newObservationAttribute))
logger.info("Loaded Min-Values:\t" + str(newMinValues))
logger.info("Loaded Max-Values:\t" + str(newMaxValues))

INFO:__main__:Loaded Training attrs:	[u'avgpressure', u'avgmotor', u'avgspindle', u'duration']
INFO:__main__:Loaded Observation attr:	[u'hasanomaly']
INFO:__main__:Loaded Min-Values:	[7.815327713140493, 849.0821682136137, 551.0287156231697, 10.01059556007385]
INFO:__main__:Loaded Max-Values:	[24.4976786018849, 2223.4170737832915, 2158.418250804427, 22.64933466911316]


In [114]:
deserialized_net = gluon.nn.SymbolBlock.imports(structureFile, ['data'], paramFile)

## Predictions with a loaded model
Finally, we can use the model as we did before. Let´s re-play the two artificial examples from above, but instead of the original NN, we now use the deserialized NN:

In [115]:
__makePrediction(normalizedInputData=avgData, net=deserialized_net)


[0.]
<NDArray 1 @cpu(0)>

In [116]:
makePrediction(rawInputData=maxFeatures, net=deserialized_net)


[1.]
<NDArray 1 @cpu(0)>

As expected, the first example returns a 0 (normal) prediction, because it used the averages for the sensor input which are within the normal bounds of the drilling operation. The second example uses extreme values and therefore the model predicts an anomaly.

To show that we can still normalize values, let´s create a new, artificial dataset, normalize it and then make a prediction:

In [117]:
pressure = 7.815328, # edit here if you want
motor = 849.082168, # edit here if you want
spindle = 551.028716, # edit here if you want
duration = 10.010596, # edit here if you want

isNormal(pressure=pressure, # this is the arbitrary pressure value from above
         motor=motor, # this is the arbitrary motor rpm value from above
         spindle=spindle, # this is the arbitrary spindle rpm value from above
         duration=duration, # this is the arbitrary duration value from above
         minValues=newMinValues, # the loaded min values
         maxValues=newMaxValues, # the loaded max values
         net=deserialized_net) # the deserialized NN

False

# Storing the model in S3
To make use of the model in Greengrass, we need to store it as a file in S3. The following code will compress the three parts into a single file and then push that to a S3 bucket.
***Important*** This part will fail to run on your machine if the following conditions are not met:
1. Editing the code and replacing `iiotws3-iotwss3bucket-btgokikhcqx8` with the name of the bucket in the account you are using. (If you booted this Notebook through th workshop CloudFormation template this is already done for you)
2. If you are running this from Cloud9 or another EC2 instance, this resource needs to have a role that allows writing to the aforementioned S3 bucket.
3. If you are running this from a local PC, you need to have access key credentials installed that allow writing to the S3 bucket.

In [118]:
store_model_on_s3 = True
compressed_model_file_name = "DrillingPrediction.model.tar.gz"
amazon_s3_bucket_for_model_storage = "iiotws-iiotwsgreengrass-yu8stn8r1q3p"
amazon_s3_key_for_compressed_model = "model/"+compressed_model_file_name

if store_model_on_s3:
    with tarfile.open(compressed_model_file_name, "w:gz") as tar:
        tar.add(paramFile, arcname=os.path.basename(paramFile))
        tar.add(structureFile, arcname=os.path.basename(structureFile))
        tar.add(normalizationFile, arcname=os.path.basename(normalizationFile))
    s3 = boto3.resource('s3')
    logger.info("load model to s3://{}/{}".format(amazon_s3_bucket_for_model_storage, amazon_s3_key_for_compressed_model))
    s3.Bucket(amazon_s3_bucket_for_model_storage).upload_file(compressed_model_file_name, amazon_s3_key_for_compressed_model)


INFO:__main__:load model to s3://iiotws-iiotwsgreengrass-yu8stn8r1q3p/model/DrillingPrediction.model.tar.gz
